# How to use this notebook

**Before running this notebook:**
<br>
Set up the following file structure (namely, the `raw/`, `processed/`, and `archive/` folders within `data/`). Populate `raw/` with the data that we have been given. 

```
data/
├── raw/
│   ├── district.json
│   ├── 2024-09-11_District_A_Benchmarks.json
│   ├── 2024-09-11_District_A_Scores.json
│   └── ... (future raw data files)
├── processed/
└── archive/
README.md
```

**When you run this notebook, it will...**
- Rename the raw files to follow a consistent format
- Pull out each individual table 
- Save each table as a `.csv` in `/data/processed`

All names will be formatted as follows: `[district]_[table]_[date].[extension]`
- e.g. `a_all_2024-09-11.json`
- e.g. `a_scores_2024-09-11.csv`

**After running this notebook:**
<br>
You can import tables indiviudally by using `pd.read_csv()`.


In [2]:
import os
import json
import pandas as pd

In [3]:
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(repo_root, 'predicting-proficiency', 'data')

## Renaming files

In [4]:
file_names = [
    ('2024-09-11 District A Benchmarks.json', 'a_benchmarks_2024-09-11.json'),
    ('2024-09-11 District A Scores.json', 'a_scores_2024-09-11.json'),
    ('2024-09-11 District A Vendor Student Usage.json', 'a_vendorUsage_2024-09-11.json'),
    ('district.json', 'a_all_2024-08-29.json'),
    ('vendorProducts_202409111049.csv', 'a_vendorKey_2024-09-11.csv')
]

for old_name, new_name in file_names:
    old_path = os.path.join(data_path, 'raw', old_name)
    new_path = os.path.join(data_path, 'raw', new_name)
    if os.path.exists(old_path) and not os.path.exists(new_path):
        os.rename(old_path, new_path)

In [5]:
with open('data/raw/a_all_2024-08-29.json', 'r') as file:
    data = json.loads(file.read())

with open('data/raw/a_scores_2024-09-11.json', 'r') as file:
    scores_json = json.loads(file.read())

with open('data/raw/a_benchmarks_2024-09-11.json', 'r') as file:
    benchmarks_json = json.loads(file.read())

with open('data/raw/a_vendorUsage_2024-09-11.json', 'r') as file:
    vendorUsage_json = json.loads(file.read())

## Saving .csvs

In [6]:
scores_data = scores_json[list(scores_json.keys())[0]]
scores_df = pd.DataFrame(scores_data)

benchmarks_df = pd.DataFrame(benchmarks_json['benchmarks'])
schools_df = pd.DataFrame(data['schools'])
courseSections_df = pd.DataFrame(data['courseSections'])
courseSectionRosters_df = pd.DataFrame(data['courseSectionRosters'])

vendorUsage_data = vendorUsage_json[list(vendorUsage_json.keys())[0]]
vendorUsage_df = pd.DataFrame(vendorUsage_data)

vendorKey_df = pd.read_csv('data/raw/a_vendorKey_2024-09-11.csv')

In [7]:
data_path = os.path.join(repo_root, 'predicting-proficiency', 'data')

processed_dir = os.path.join(data_path, 'processed')
if os.path.exists(processed_dir):
    for file in os.listdir(processed_dir):
        file_path = os.path.join(processed_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)

os.makedirs(os.path.dirname(data_path), exist_ok=True)
benchmarks_df.to_csv(os.path.join(data_path, 'processed', 'a_benchmarks_2024-09-11.csv'), index=False)
schools_df.to_csv(os.path.join(data_path, 'processed', 'a_schools_2024-08-29.csv'), index=False)
courseSections_df.to_csv(os.path.join(data_path, 'processed', 'a_courseSections_2024-08-29.csv'), index=False)
courseSectionRosters_df.to_csv(os.path.join(data_path, 'processed', 'a_courseSectionRosters_2024-08-29.csv'), index=False)
scores_df.to_csv(os.path.join(data_path, 'processed', 'a_scores_2024-09-11.csv'), index=False)
vendorUsage_df.to_csv(os.path.join(data_path, 'processed', 'a_vendorUsage_2024-09-11.csv'), index=False)

You can stop here! The rest of the notebook is just flagging issues in the data that need to be resolved.

## Potential issues

We currently have two DataFrames with info on vendors: `vendorUsage_df` which contains usage stats with numerical ids for each vendor, and `vendorKey_df` which is a lookup table containing the name and id of each vendor. We will merge the two and export it as a single csv.

Actually we won't yet because the ids don't match up.

In [8]:
# vendorUsage_df = vendorUsage_df.merge(
#     vendorKey_df[['vendorId', 'productName']], 
#     on='vendorId', 
#     how='left'
# )

In [9]:
vendorUsage_df['vendorId'].unique()

array([72609, 11333, 39195, 27172, 61673, 37017, 36130, 62382, 62509,
       62508, 37269, 25405, 52447, 52448, 61544,  7273, 63431, 63432,
       22410, 63616, 23165,  7035, 11464])

In [10]:
vendorKey_df['vendorId'].unique()

array([11361, 21186, 21143, 11333, 13895, 11272, 11179, 72615, 14726,
       72616, 72617, 72606, 72618, 72619, 14617, 13793, 72622, 11464,
       14026, 11281])

In [11]:
set1 = set(vendorUsage_df['vendorId'].unique())
set2 = set(vendorKey_df['vendorId'].unique())

missing_keys = set1 - set2
num_missing = len(missing_keys)

print(f"Number of vendorIds in vendorUsage with no key: {num_missing}")
print(f"VendorIds with no key: {missing_keys}")

Number of vendorIds in vendorUsage with no key: 21
VendorIds with no key: {np.int64(63616), np.int64(22410), np.int64(37269), np.int64(37017), np.int64(39195), np.int64(72609), np.int64(36130), np.int64(27172), np.int64(62508), np.int64(62509), np.int64(62382), np.int64(25405), np.int64(63431), np.int64(63432), np.int64(52447), np.int64(52448), np.int64(61544), np.int64(61673), np.int64(7273), np.int64(7035), np.int64(23165)}


In [12]:
dfList = [benchmarks_df, schools_df, courseSections_df, courseSectionRosters_df, scores_df, vendorKey_df, vendorUsage_df]

Benchmarks

In [19]:
benchmarks_df.head(10)

,districtId,year,level,dataSource,value
0,45,2017,3,value_scantronMath,2356.0
1,45,2017,3,value_scantronReading,2563.0
2,45,2017,4,value_scantronMath,2493.0
3,45,2017,4,value_scantronReading,2674.0
4,45,2017,5,value_scantronMath,2584.0
5,45,2017,5,value_scantronReading,2841.0
6,45,2017,6,value_scantronMath,2675.0
7,45,2017,6,value_scantronReading,2897.0
8,45,2017,7,value_scantronMath,2766.0
9,45,2017,7,value_scantronReading,3008.0


In [15]:
benchmarks_df.shape

(42, 5)

In [17]:
benchmarks_df.dataSource.unique()

array(['value_scantronMath', 'value_scantronReading',
       'value_actCompositeScore', 'value_aspireReadingScaleScore',
       'value_starMath', 'value_actMathScore', 'value_actReadingScore',
       'value_actEnglishScore', 'value_actScienceScore'], dtype=object)

Scores

In [22]:
scores_df.head(10)

,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ethnicity,subgroup_ell,subgroup_bottom25,...,value_starReading,value_starMath,value_ScienceMilestones,value_LEAPScience,value_SocialMilestones,value_gcaInfo,value_gcaNar,value_gcaOpin,value_aimsWebMath,value_aimsWebReading
0,44774,9,2018,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,44774,8,2017,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,44774,8,2017,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44775,8,2017,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44775,8,2017,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,44775,9,2018,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,44776,9,2018,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,44776,8,2017,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,44776,8,2017,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,44777,6,2017,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
scores_df.shape

(303750, 69)

In [27]:
scores_df.studentLevel.unique()

array([ 9,  8,  6,  7,  5, 12, 11, 10,  4,  3,  2,  1,  0, 99, 98, 97, 96,
       -1])

In [26]:
scores_df.columns

Index(['studentId', 'studentLevel', 'year', 'measurementTypeId',
       'subgroup_specialEd', 'subgroup_lunchStatus', 'subgroup_gender',
       'subgroup_ethnicity', 'subgroup_ell', 'subgroup_bottom25',
       'value_actMathScore', 'value_actEnglishScore', 'value_actReadingScore',
       'value_actScienceScore', 'value_actCompositeScore', 'value_nweaMath',
       'value_nweaReading', 'value_nweaEnglish', 'value_iReadyMath',
       'value_iReadyReading', 'value_actMasteryComposite',
       'value_actMasteryReading', 'value_actMasteryMath',
       'value_actMasteryScience', 'value_actMasteryEnglish', 'value_nweaELA',
       'value_scantronMath', 'value_scantronReading', 'value_mapMath',
       'value_mapEnglish', 'value_mapReading', 'value_mapRLA',
       'value_scantronScience', 'value_scantronLanguage',
       'value_ReadingFoundations', 'value_scantronAlgebra', 'value_eocMath',
       'value_actELAScore', 'value_mapScience', 'value_LeapELAScale',
       'value_LeapMathScale', 'value_L

Note: We can potentially narrow down the above list by looking at which types of tests/grade levels we have benchmarks for

In [25]:
scores_df.describe()

,studentId,studentLevel,year,measurementTypeId,subgroup_specialEd,subgroup_lunchStatus,subgroup_gender,subgroup_ethnicity,subgroup_ell,subgroup_bottom25,...,value_starReading,value_starMath,value_ScienceMilestones,value_LEAPScience,value_SocialMilestones,value_gcaInfo,value_gcaNar,value_gcaOpin,value_aimsWebMath,value_aimsWebReading
count,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,303750.000000,...,303750.000000,303750.000000,303750.000000,303750.0,303750.000000,303750.0,303750.0,303750.0,303750.0,303750.0
mean,131440.302008,6.004998,2017.208234,2.757402,0.320174,0.256807,0.251562,0.793995,0.000040,0.016823,...,4.174281,86.041310,26.027691,0.0,26.343312,0.0,0.0,0.0,0.0,0.0
std,57982.885934,5.916790,10.375606,1.281897,6.673339,1.104788,1.134392,3.593934,0.010887,0.128608,...,48.159151,446.624951,113.742834,0.0,113.992499,0.0,0.0,0.0,0.0,0.0
min,44774.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
25%,85484.000000,4.000000,2017.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
50%,121078.000000,6.000000,2017.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
75%,169950.000000,8.000000,2018.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
max,259514.000000,99.000000,2020.000000,7.000000,1000.000000,6.000000,6.000000,18.000000,3.000000,1.000000,...,1336.000000,3247.000000,820.000000,0.0,765.000000,0.0,0.0,0.0,0.0,0.0
